In [ ]:
include("setup.jl")

# Asymmetric Travelling Salesman Problem

https://www.movable-type.co.uk/scripts/latlong.html
### Problem description

$$
    \begin{array}{cl}
        \min_{x} \ \ \ &
            \sum_{i, j} d_{i, j} x_{i, j}\\
        s.t. &
            \sum_{j} x_{i, j} = 1, \ \ \ \forall i \in \Omega\\
        &    \sum_{i} x_{i, j} = 1, \ \ \ \forall j \in \Omega\\
        &    x_{k,k} = 0 \ \ \ \forall k \in \Omega\\
        &   \sum_{i} \sum_{j \ne i} x_{i,j} \le |S| - 1, \forall S \subset \Omega, 2 \le |S| \le N-1\\
        &   x_{i, j} \in \{0, 1\}, \ \ \ \forall (i,j) \in \Omega^2 , i \ne j\\
        & \Omega = \{1, \dots, N\}
    \end{array}
$$

In [ ]:
include("tsp/DataReader.jl")

In [ ]:
d = load(string(workspace,"/tsp/distances_quebec.jld"),"distances")

In [ ]:
c = load(string(workspace,"/tsp/cities_quebec.jld"),"cities")

In [ ]:
"""
    all_subsets(x)

Compute all subsets of elements of vector `x`.
"""
function all_subsets(x::Vector{T}) where T
    res = Vector{T}[[]]  # Vector of vectors
    for elem in x, j in eachindex(res)
        push!(res, [res[j] ; elem])
    end
    return res
end

In [ ]:
all_subsets([1:5;])

In [ ]:
"""
    solve_tsp(D, optimizer)

Compute a shortest TSP tour given then matrix distance `D`.
"""
function solve_tsp(D, optimizer)
    # Number of cities
    N = size(D, 1)
    N == size(D, 2) || throw(DimensionMismatch())                     # sanity check: `D` is square
    N <= 16 || error("N cannot be larger than 16 for memory safety")  # sanity check: `N` is not too large
    
    # Instantiate a model
    mip = Model(with_optimizer(optimizer))

    # I. Create arc variables
    @variable(mip, X[1:N, 1:N], Bin)

    # II. Set objective
    # TODO

    # III. Add constraints to the model
    
    # III.1 
    # No city can be its own follower in the tour
    # TODO

    # III.2
    # Each city has one predecessor and one successor
    # TODO
    
    # III.3
    # Sub-tour elimination constraints
    # TODO

    # Solve MIP model
    optimize!(mip)
    
    println("Optimal tour length is ", objective_value(mip))

    # Return solution
    return value.(X)
end

In [ ]:
solve_tsp(d, Cbc.Optimizer)

In [ ]:
sol = solve_tsp(d, GLPK.Optimizer)

In [ ]:
include("tsp/RoadTrip.jl")

In [ ]:
RoadTrip(9, c, sol)

In [ ]:
d2 = load(string(workspace,"/tsp/distances_usa.jld"),"distances2")

In [ ]:
c2 = load(string(workspace,"/tsp/cities_usa.jld"),"cities2")

In [ ]:
"""
    find_subtours(X)

Compute sub-tours from tentative solution `X`.
"""
function find_subtours(X; verbose=false)
    
    N = size(X, 1)
    N == size(X, 2) || throw(DimensionMismatch())  # sanity check

    # List of all sub-tours in `X`
    # Empty if `X` is a tour
    sub_tours = Vector{Vector{Int}}()

    # Compute all sub-tours
    # TODO
    
    verbose && println("Found $(length(sub_tours)) sub-tours.")
    return sub_tours

end

In [ ]:
# Some basic unit tests
# If a Test fails, there is an error in your implementation
# Passing the tests does not mean your implementation is correct!
@testset begin
    _s1 = find_subtours(
        [
            [0.0 1.0];
            [1.0 0.0]
        ]
    )  # no sub-tour should be returned here

    _s2 = find_subtours(
        [
            [0.0 1.0 0.0 0.0];
            [1.0 0.0 0.0 0.0];
            [0.0 0.0 0.0 1.0];
            [0.0 0.0 1.0 0.0]
        ]
    )  # two sub-tours should be returned here
    
    @test length(_s1) == 0
    @test length(_s2) == 2
end;

In [ ]:
"""
    add_subtour_elimination!(mip, X, s)

Add sub-tour elimination constraint of sub-tour `s`.
"""
function add_subtour_elimination!(mip, X, s)
    
    # TODO

    return nothing
end

In [ ]:
function solve_tsp2(n, D, optimizer; itermax = 100)

    # instantiate model
    mip = Model(with_optimizer(optimizer))

    # Basic formulation
    # TODO

    t_start = time()
    num_iter = 0
    num_subtours = 0
    @printf("%4s %8s %8s %8s\n", "Iter", "LB", "#tours", "Time")
    
    while (time() - t_start < 300.0) && (num_iter < itermax)  # stopping conditions to be safe
        num_iter += 1
        
        # solve relaxed model
        optimize!(mip)
        # get current objective
        objval = objective_value(mip)
        
        # Log
        @printf("%4d %8d %8d %8.2f\n", num_iter, objval, num_subtours, time() - t_start)

        # get solution
        X_ = value.(X)

        # find sub-tours
        sub_tours = find_subtours(X_)
        num_subtours += length(sub_tours)

        if length(sub_tours) == 0
            @printf "Optimal solution found.\n\nOptimal tour length: %8d" objval
            return X_
        else
            for subtour in sub_tours
                add_subtour_elimination!(mip, X, subtour)
            end
        end
    end
    
    @info "Stopping criterion reached."
    return value.(X)
end

In [ ]:
sol2 = solve_tsp2(50, d2, GLPK.Optimizer)

In [ ]:
RoadTrip(50, c2, sol2)

![](tsp/solution_usa.png)